## Scopes, Closures and Decorators

- [**Scopes**](#scopes)
- [**Closures**](#closures)
- [**Decorators**](#decorators)

---

### Scopes <a name='scopes'></a>

Python has 4 major types of variable scopes:

- `Local`: The scope inside functions, variables defined inside the function are assigned to that scope. It has access to both global and built-in scopes.
- `*Enclosing`: For nested functions, inner functions have access to the enclosing scope (i.e. the scope of their outer functions).
- `Global`: The scope inside a module, it has access to built-in scope.
- `Built-in`: Widest scope where all special reserved keywords fall under it.

If a variable has not been found under the current scope, Python will search one layer above each time until it finds it, otherwise it will throw an exception.

<font color='orange'> Caveat: Since Python searches variables follow L $\rightarrow$ E $\rightarrow$ G $\rightarrow$ B rule, if there is any function written in global scope sharing the same name of reserved one in built-in scope, only the self-written one will be used throught the program. </font>

> `Global variable` & `local variable`:

In [1]:
# Case 1: count -> nonlocal at compile time
# Call the global variable inside a function without declaring or modifying, so it is just referencing the global variable
count = 0

def test():
    print(count)

test()

0


In [2]:
# Case 2: count -> local at compile time (inside function), count -> global at compile time (outside function)
# When there is an assignment or modification inside function, the variable will be considered as local
count = 0

def test():
    count = 1
    print(count)
    
test()
print(count)

1
0


In [3]:
# Case 3: count -> local at compile time (inside function), count -> global at compile time (outside function)
# When there is an assignment or modification inside function, the variable will be considered as local, therefore if 
# calling is prior to declaration, an exception will be thrown
count = 0

def test():
    print(count)
    count = 1
    
test()

UnboundLocalError: local variable 'count' referenced before assignment

In [4]:
# Case 4: count -> global at compile time
# Declare explicitly that the variable inside function is global, then modify the variable globally
count = 0

def test():
    global count
    count += 1
    print(count)
    
test()
print(count)

1
1


> `Non-local variable`: Similarly as global cases, but non-local is used for inner functions to modify the variables declared in outer function scope.

In [5]:
# Without nonlocal keyword
def outer():
    x = 'hello'
    def inner():
        x = 'world'
    inner()
    print(x)
outer()

hello


In [6]:
# With nonlocal keyword, the outer variable can be modified in inner scope
def outer():
    x = 'hello'
    def inner():
        nonlocal x
        x = 'world'
    inner()
    print(x)
outer()

world


---

### [Closures](https://www.pythontutorial.net/advanced-python/python-closures/) <a name=closures></a>

In nested functions, when a variable is called or referenced in inner function while it is only declared in the outer function (i.e. the variable is accessed from its nonlocal scope), it is called **free variable**, which is `x` in the example below:

In [7]:
def outer():
    x = 'hello'
    def inner():
        print(x)
    return inner  # Note the function is returned without parenthesis, which is necessary for closure to work

fn = outer()
fn()

hello


In [8]:
fn.__closure__

(<cell at 0x000001A385A39B50: str object at 0x000001A385A34A70>,)

In the example, when looking at the `inner` function, we are actually looking at:
- The `inner` function itself.
- The free variable `x` with the value 'hello'.

The combination of this two is called a **closure**. Through the use of closure, even when the free variable is destroyed after the execution of `outer` function, it is still accessible in returned function.

---

### Decorators <a name='decorators'></a>

> `Definition`: Decorators are used to modify the behaviour of function or class, it:
> - takes a function as an argument
> - returns a closure which usually accepts any combination of parameters

In [9]:
def counter(fn):
    count = 0
    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print('Function {} was called {} times'.format(fn.__name__, count))
        return fn(*args, **kwargs)
    return inner

In [10]:
def add(a, b):
    return a+b

In [11]:
# Way 1: decoration
add = counter(add)
add(1, 2)

Function add was called 1 times


3

In [12]:
# Way 2: decoration
@counter
def subtract(a, b):
    return a-b

subtract(2, 1)

Function subtract was called 1 times


1

> `Decorator introspection`:
> After decoration, the returned function is not the same function as before after all, therefore the docstring and name of initial function are lost.

In [13]:
print(subtract.__name__)

inner


In [14]:
# Use wraps to keep information of the original function
from functools import wraps

In [15]:
def counter(fn):
    count = 0
    
    # Decorate closure function with wrap
    @wraps(fn)
    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print('Function {} was called {} times'.format(fn.__name__, count))
        return fn(*args, **kwargs)
    return inner

In [16]:
@counter
def mul(a, b):
    return a*b

print(mul.__name__)

mul


> `Decorator stacking`: Simply add multiple @decorator on top of the function, but notice the order of decorators matters.
```python
    @dec1
    @dec2
    @dec3
    def func():
        pass
```

> `Parameterized decorator`: Use nested closure, i.e. a decorator factory outside the original decorator to pass in arguments into the decorator.

> `Decorator application examples`:

* logger

In [17]:
def logger(fn):
    from functools import wraps
    from datetime import datetime, timezone
    
    @wraps(fn)
    def inner(*args, **kwargs):
        run_dt = datetime.now(timezone.utc)
        result = fn(*args, **kwargs)
        print('{}: called {}'.format(run_dt, fn.__name__))
        
        return result
    return inner

- cache

In [18]:
# Use lru_cache to store cache of computed information
from functools import lru_cache

In [19]:
# Fibonacci example
@lru_cache(maxsize=8)
def fib(n):
    print('Calculating fib({})'.format(n))
    return 1 if n<3 else fib(n-1) + fib(n-2)

In [20]:
fib(10)

Calculating fib(10)
Calculating fib(9)
Calculating fib(8)
Calculating fib(7)
Calculating fib(6)
Calculating fib(5)
Calculating fib(4)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)


55

In [21]:
# Use cached memory
fib(2)

Calculating fib(2)


1